## 데이터 불균형 해소
- 데이터 불균형 해소를 위해 클래스 제거/ 데이터 증강 및 언더샘플링을 진행합니다.
- 판단 기준을 위해 data_explore_and_viz.ipynb 파일을 참고합니다.

In [21]:
import os
import shutil

def delete_folders_with_few_images(base_dir, threshold=50):
    """임계값보다 적은 파일 수를 보유한 클래스 제거"""
    # 기본 디렉토리의 각 주요 폴더를 순회
    for main_folder in os.listdir(base_dir):
        main_folder_path = os.path.join(base_dir, main_folder)
        if os.path.isdir(main_folder_path):  # 폴더인지 확인
            # 주요 폴더의 각 서브폴더를 확인
            for subfolder in os.listdir(main_folder_path):
                subfolder_path = os.path.join(main_folder_path, subfolder)
                if os.path.isdir(subfolder_path):
                    # 이 서브폴더에서 모든 jpg 파일을 나열
                    files = [f for f in os.listdir(subfolder_path) if f.endswith('.jpg')]
                    file_count = len(files)
                    
                    # 파일 수가 임계값보다 적으면 서브폴더를 삭제
                    if file_count < threshold:
                        for f in os.listdir(subfolder_path):
                            os.chmod(os.path.join(subfolder_path, f), 0o666) # 파일에 대한 권한 부여
                        shutil.rmtree(subfolder_path)  # 폴더와 그 내용을 모두 삭제
                        print(f"Deleted folder: {subfolder} in {main_folder} with {file_count} files.")

# 정리하고자 하는 디렉토리 경로
base_dir = "angle_jpg4"
delete_folders_with_few_images(base_dir)



Deleted folder: 기아_쏘울_19 in car_back with 32 files.
Deleted folder: 현대_넥쏘_2020 in car_back with 14 files.
Deleted folder: 현대_벨로스터_17 in car_back with 13 files.
Deleted folder: 현대_아이오닉_21 in car_back with 25 files.
Deleted folder: 현대_코나_21 in car_back with 42 files.
Deleted folder: 기아_쏘울_19 in car_front with 27 files.
Deleted folder: 현대_넥쏘_2020 in car_front with 13 files.
Deleted folder: 현대_벨로스터_17 in car_front with 15 files.
Deleted folder: 현대_아이오닉_21 in car_front with 25 files.
Deleted folder: 현대_코나_21 in car_front with 29 files.
Deleted folder: 기아_쏘울_19 in car_side with 33 files.
Deleted folder: 현대_넥쏘_2020 in car_side with 43 files.
Deleted folder: 현대_벨로스터_17 in car_side with 10 files.
Deleted folder: 현대_아이오닉_21 in car_side with 27 files.
Deleted folder: 현대_코나_21 in car_side with 29 files.


In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from random import sample

def adjust_data(folder, target_min, target_max):
    """
    주어진 폴더의 각 클래스에 대해 데이터를 증강하거나 감소시켜, 
    지정된 최소 및 최대 파일 수에 맞추어 조정
    """
    # 데이터 증강 설정
    datagen = ImageDataGenerator(
        rotation_range=10,  # 0-10도 사이에서 이미지 회전
        width_shift_range=0.10,  # 이미지를 수평으로 최대 10% 이동
        brightness_range=(0.8, 1.2),  # 80%에서 120% 사이의 밝기 조절
        height_shift_range=0.10,  # 이미지를 수직으로 최대 10% 이동
        fill_mode='nearest',  # 이동할 때 가장자리 픽셀로 채움
        horizontal_flip=True,  # 이미지를 수평으로 뒤집기
        rescale=1./255  # 이미지 픽셀 값을 0-1 범위로 조정
    )
    classes = os.listdir(folder)
    for cls in classes:
        class_dir = os.path.join(folder, cls)
        files = os.listdir(class_dir)
        num_files = len(files)
        
        if num_files < target_min:
            # 오버샘플링: 데이터 증강을 통해 새 파일 생성
            needed_copies = target_min - num_files
            while needed_copies > 0:
                for i in range(min(needed_copies, num_files)):  # 필요한 복사 수를 초과하지 않도록 함
                    original_file = files[i % num_files]
                    img_path = os.path.join(class_dir, original_file)
                    img = image.load_img(img_path, target_size=(224, 224))  # 모델이 예상하는 입력 크기로 이미지 크기 조정
                    x = image.img_to_array(img)
                    x = x.reshape((1,) + x.shape)
                    
                    # 아래 .flow() 명령은 무작위로 변형된 이미지의 배치를 생성하고 결과를 'class_dir' 디렉토리에 저장함
                    for batch in datagen.flow(x, batch_size=1, save_to_dir=class_dir, save_prefix='aug', save_format='jpg'):
                        needed_copies -= 1
                        break  # 무한 루프를 피하기 위해 첫 이미지 후에 중단
                    if needed_copies <= 0:
                        break

        elif num_files > target_max:
            # 언더샘플링: 파일을 무작위로 제거
            files_to_remove = sample(files, num_files - target_max)
            for file in files_to_remove:
                os.chmod(os.path.join(class_dir, file), 0o666)
                os.remove(os.path.join(class_dir, file))

# 각 각도에 대한 조정 적용
adjust_data('angle_jpg4/car_front', 250, 250)
adjust_data('angle_jpg4/car_back', 250, 250)
adjust_data('angle_jpg4/car_side', 250, 250)

In [27]:
import os

def count_files(directory):
   file_counts = {}  # 파일 개수를 저장할 딕셔너리 초기화
   
   # 지정된 디렉토리 바로 아래에 있는 모든 하위 디렉토리 목록 생성
   subdirectories = [os.path.join(directory, d) for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
   
   # 각 하위 디렉토리에 있는 파일 개수 계산
   for subdir in subdirectories:
       files = [f for f in os.listdir(subdir) if os.path.isfile(os.path.join(subdir, f))]
       folder_name = os.path.basename(subdir)
       file_counts[folder_name] = len(files)
       
   return file_counts

directory = 'angle_jpg4/car_back'
file_counts = count_files(directory)

import pandas as pd

# 딕셔너리에서 DataFrame 생성
df = pd.DataFrame(list(file_counts.items()), columns=['Folder', 'Number of Files'])
print(df)

                      Folder  Number of Files
0                기아_K3_17-18              250
1                기아_K3_19-21              250
2                기아_K5_17_MX              249
3                기아_K5_18-19              250
4                기아_K5_20-21              249
5                기아_K7_17-19              250
6                기아_K9_18-21              250
7                기아_니로_17-18              250
8                기아_니로_19-21              250
9                기아_레이_17-21              250
10                  기아_모닝_17              250
11               기아_모닝_18-21              250
12              기아_모하비_17-19              250
13              기아_봉고3_17-21              250
14              기아_셀토스_18-21              250
15              기아_스토닉_17-21              250
16              기아_스팅어_17-21              247
17             기아_스포티지_17-18              250
18             기아_스포티지_19-21              250
19              기아_쏘렌토_17-20              250
20                 기아_쏘렌토_21      

In [41]:
# 클래스 숫자 체크
len(os.listdir('angle_jpg4/car_front'))

46

In [40]:
# 각 방향 폴더에 해당하는 클래스들이 동일한지 체크
a = os.listdir('angle_jpg4/car_front')
b = os.listdir('angle_jpg4/car_back')
c = os.listdir('angle_jpg4/car_side')

set_a = set(a)
set_b = set(b)
set_c = set(c)

set_a == set_b == set_c

True